# Tropical Cyclone Wind Estimation from Satellite Imagery

By: Ali Ahmadalipour, Principal Data Scientist at KatRisk

To check out a summary of the methodology and results, please visit https://www.linkedin.com/pulse/deep-learning-computer-vision-satellite-imagery-ali-ahmadalipour/

# 1. Import data and unzip

This section will download the data from Radiant MLHub. 

But for this presentation, we will skip running it, since I have alreay downloaded and preprocessed the data to save time during the lecture.

In [ ]:
import os
import requests
import tarfile
import urllib.request

### Download data using Radiant MLHub:

In [ ]:
# pip install radiant_mlhub

In [ ]:
from radiant_mlhub import Dataset, Collection, client
#os.environ['MLHUB_API_KEY'] = 'YOUR_MLHub_API_KEY'

In [ ]:
dataset = Dataset.fetch('nasa_tropical_storm_competition')
download_dir = Path('./').resolve()
archive_paths = dataset.download(output_dir=download_dir)
#print('Download completed')

### Check the number of files to ensure correct extraction

In [ ]:
## Count the number of files to make sure that all files are correctly loaded and extracted:
#directory ='.'
#path, dirs, files = next(os.walk(directory+'/train'))
#file_count = len(files)
#file_count #70257

# 2. Preprocessing data:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import seaborn as sns
import time
%matplotlib inline

You can access the pre-processed data from this shared Google Drive folder:

https://drive.google.com/drive/folders/1jTw5UoLewMsD0SckoyTcfbN7UNWXl9EN?usp=sharing

To load the data from the shared Google drive, open the shared drive in your browser,
right click on the folder, and select "Add shortcut to Drive". Now you should see it on your Google Drive.

In [ ]:
# mount your Google drive to this notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
directory ='/content/drive/MyDrive/Lecture_10/' 
train_metadata  = pd.read_csv(f'{directory}/training_set_features.csv')
train_labels = pd.read_csv(f'{directory}/training_set_labels.csv')
#directory = '.'

In [ ]:
train_metadata.head()

In [ ]:
train_labels.head()

In [ ]:
# to check if everything loaded fine
train_metadata.storm_id.nunique()

In [ ]:
full_metadata = train_metadata.merge(train_labels, on="image_id")

In [ ]:
full_metadata['file_name'] = full_metadata.image_id.apply(lambda x: f'{directory}train/{x}.jpg')

In [ ]:
full_metadata.head()

In [ ]:
# load a single image and see how it is:
img = mpimg.imread(f'{directory}sample_img_abs_010.jpg')
print('image shape=',img.shape)
plt.figure(figsize=[4.5,4])
plt.pcolor(img)
plt.colorbar()

In [ ]:
def resize_crop_img(img):
    # The function takes an array of a 366x366 image and converts it to a 64x64 shape.
    # The input array for this exercise should be 366x366.
    #----------------------------------------------
    new_img_size = 256
    # 1. Cropping a 256x256 size array that has the highest values (i.e the eye of cyclone and its surrounding)
    n_passes = img.shape[0] - new_img_size
    slice_avg = np.array([[img[row:row+new_img_size,col:col+new_img_size].mean() \
                           for col in range(n_passes)] for row in range(n_passes)])
    max_index = np.unravel_index(slice_avg.argmax(), slice_avg.shape)
    img_crop = img[max_index[0]:max_index[0]+new_img_size, max_index[1]:max_index[1]+new_img_size]
    # 2. Reshaping the array from 256x256 to 64x64 (same as using a MaxPool2D with size=(2,2) and stride=2)
    img_crop_resize = img_resize = img_crop.reshape(64, 4, 64, 4).max(-1).max(1)

    # another way to do the above but using tensorflow. This is much slower. 
    # One can also consider using opencv for this part, as that library is very efficient for image processing.
    """
    average_pool_2d = tf.keras.layers.AveragePooling2D(pool_size=(new_img_size, new_img_size), 
                                                       strides=(1, 1))
    max_pool_2d = tf.keras.layers.MaxPool2D(pool_size=(4, 4), 
                                                       strides=(1, 1))
    x = tf.constant(np.trunc(img))
    slice_avg = average_pool_2d(tf.reshape(x, [1, 366, 366, 1]))
    slice_avg = np.array(slice_avg[0,:,:,0]).reshape(111,111)
    img_crop = x[max_index[1]:max_index[1]+new_img_size, max_index[2]:max_index[2]+new_img_size]
    img_crop_resize = max_pool_2d(tf.reshape(x, [1, 256, 256, 1]))
    """

    return img_crop_resize

In [ ]:
# load a single image and see how the resizing works:
img = mpimg.imread(f'{directory}/sample_img_abs_010.jpg')
print('Original image shape=',img.shape)
plt.figure(figsize=[4.5,4])
plt.pcolor(img, cmap='RdYlBu')
plt.colorbar()
plt.figure(figsize=[4.5,4])
plt.pcolor(resize_crop_img(img), cmap='RdYlBu')
print('Resized image shape=',resize_crop_img(img).shape)
plt.colorbar();

In [ ]:
# Add a temporary column for number of images per storm
images_per_storm = full_metadata.groupby("storm_id").size().to_frame("images_per_storm")
full_metadata = full_metadata.merge(images_per_storm, how="left", on="storm_id")

In [ ]:
# Each storm is sorted by relative time
# Identify the final 20% of images per storm
full_metadata["pct_of_storm"] = (
    full_metadata.groupby("storm_id").cumcount() / full_metadata.images_per_storm
)
train = full_metadata[full_metadata.pct_of_storm < 0.8].drop(
    ["images_per_storm", "pct_of_storm"], axis=1
)
val = full_metadata[full_metadata.pct_of_storm >= 0.8].drop(
    ["images_per_storm", "pct_of_storm"], axis=1
)

In [ ]:
train.head(2)

# 3. Resizing training data (only done if limited memory):

In [ ]:
# This part is only done to reduce input file size and to be able to run the code using limited memory 
# on Google Colab. If you are not limited with computation power, you can skip sections 3 & 4.
# The resulting files from section 3 and 4 are already saved in the shared Google Drive

In [ ]:
def extract_data_chunk(i, period):
    chunk_size = 1000
    if period == 'train':
        if i == len(train)//chunk_size:
            x_train = np.stack(([resize_crop_img(mpimg.imread(train.file_name.iloc[x])/255) \
                                 for x in range(chunk_size*i,len(train))]),
                               axis=2).transpose(2, 0, 1).reshape(chunk_size*i-len(train),64,64,1)
        else:
            x_train = np.stack(([resize_crop_img(mpimg.imread(train.file_name.iloc[x])/255) \
                                for x in range(chunk_size*i,chunk_size*(i+1))]),
                               axis=2).transpose(2, 0, 1).reshape(chunk_size,64,64,1)
        np.save(f'{directory}/chunks/x_train_part_{i:03.0f}.npy',
                x_train, allow_pickle=False)

    elif period == 'val':
        if i == len(val)//chunk_size:
            x_val = np.stack(([resize_crop_img(mpimg.imread(val.file_name.iloc[x])/255) \
                               for x in range(chunk_size*i,len(val))]),
                             axis=2).transpose(2, 0, 1).reshape(chunk_size*i-len(val),64,64,1)
        else:
            x_val = np.stack(([resize_crop_img(mpimg.imread(val.file_name.iloc[x])/255) \
                                for x in range(chunk_size*i,chunk_size*(i+1))]),
                             axis=2).transpose(2, 0, 1).reshape(chunk_size,64,64,1)
        np.save(f'{directory}/chunks/x_val_part_{i:03.0f}.npy',
                x_val, allow_pickle=False)

In [ ]:
for i in range(14):
    extract_data_chunk(i, period='val')
for i in range(57):
    extract_data_chunk(i, period='train')

In [ ]:
# Stacking images
dir_str = f'{directory}chunks'
x_train = np.concatenate(([np.load(f'{dir_str}/x_train_part_{i:03.0f}.npy') for i in range(57)]),
                    axis=0)
x_val = np.concatenate(([np.load(f'{dir_str}/x_val_part_{i:03.0f}.npy') for i in range(14)]),
                    axis=0)

In [ ]:
x_train.shape, x_val.shape

In [ ]:
len(train), len(val)

In [ ]:
y_train = train.wind_speed.iloc[:len(x_train)].values
y_val = val.wind_speed.iloc[:len(x_val)].values

In [ ]:
y_train.shape, y_val.shape

In [ ]:
#np.save(f'{directory}x_train.npy', x_train, allow_pickle=False)
#np.save(f'{directory}x_val.npy', x_val, allow_pickle=False)

# 4. Reshaping test data (only done if limited memory):

In [ ]:
test_metadata  = pd.read_csv(f'{directory}test_set_features.csv')

In [ ]:
test_metadata['file_name'] = test_metadata.image_id.apply(lambda x: f'{directory}/test/{x}.jpg')
test_metadata.head(3)

In [ ]:
len(test_metadata)

In [ ]:
def extract_testdata_chunk(i, test_metadata):
    chunk_size = 1000
    if i == len(test_metadata)//chunk_size:
        x_test = np.stack(([resize_crop_img(mpimg.imread(test_metadata.file_name.iloc[x])/255) \
                              for x in range(chunk_size*i,len(test_metadata))]),
                            axis=2).transpose(2, 0, 1).reshape(chunk_size*i-len(test_metadata),64,64,1)
    else:
        x_test = np.stack(([resize_crop_img(mpimg.imread(test_metadata.file_name.iloc[x])/255) \
                            for x in range(chunk_size*i,chunk_size*(i+1))]),
                            axis=2).transpose(2, 0, 1).reshape(chunk_size,64,64,1)
    np.save(f'{directory}/chunks/x_test_part_{i:03.0f}.npy',
            x_test, allow_pickle=False)

In [ ]:
for i in range(45):
    start_time = time.time()
    extract_testdata_chunk(i, test_metadata)
    print(f'Part {i} is done!')
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Stacking images
dir_str = f'{directory}/chunks'
x_test = np.concatenate(([np.load(f'{dir_str}/x_test_part_{i:03.0f}.npy') for i in range(45)]),
                    axis=0)

In [ ]:
x_test.shape

In [ ]:
#np.save(f'{directory}x_test.npy', x_test, allow_pickle=False)

# 5. Rotating and duplicating low frequency images

'''
To load the data from the shared Google drive, open the shared drive, right click on the folder, and select "Add shortcut to Drive" and select My Drive. You will then be able to access and load the data in Google Colab.
'''

In [ ]:
x_train = np.load(f'{directory}x_train.npy')
x_val = np.load(f'{directory}x_val.npy')
y_train = train.wind_speed.iloc[:len(x_train)].values
y_val = val.wind_speed.iloc[:len(x_val)].values

In [ ]:
x_train.shape, x_val.shape

In [ ]:
y_train.shape, y_val.shape

In [ ]:
# For data augmentation and rotating input images, I am only focusing on the images corresponding to wind speeds
# above 40. The value 40 is an arbitrary value to make sure that the code is still able to be run on Google Colab.
train_highwind = train.reset_index(drop=True)[train.reset_index(drop=True).wind_speed>40]
#train_highwind.head()

In [ ]:
x_highwind = x_train[train_highwind.index,:,:,:]
y_highwind = y_train[train_highwind.index]
x_highwind_rot1 = np.rot90(x_highwind, axes=(1,2))
x_highwind_rot2 = np.rot90(x_highwind, 2, axes=(1,2))
x_highwind_rot3 = np.rot90(x_highwind, 3, axes=(1,2))

In [ ]:
# Sample plot for rotating: 
plt.figure(figsize=[15,3.5])
plt.subplot(141)
plt.pcolor(x_highwind[0,:,:,0], cmap='RdYlBu')
plt.subplot(142)
plt.pcolor(x_highwind_rot1[0,:,:,0],cmap='RdYlBu')
plt.subplot(143)
plt.pcolor(x_highwind_rot2[0,:,:,0],cmap='RdYlBu')
plt.subplot(144)
plt.pcolor(x_highwind_rot3[0,:,:,0],cmap='RdYlBu')

In [ ]:
x_train_extra = np.concatenate((x_train,x_highwind_rot1,x_highwind_rot2,x_highwind_rot3), axis=0)
y_train_extra = np.concatenate((y_train,y_highwind,y_highwind,y_highwind), axis=0)

In [ ]:
x_train.shape, x_train_extra.shape

In [ ]:
y_train.shape, y_train_extra.shape

# 6. Developing the CNN model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping#, ModelCheckpoint
#from tensorflow.keras.optimizers import SGD
from functools import partial
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
alpha = 0.0001
padding = 'same' #'valid'
activation = tf.keras.layers.LeakyReLU(alpha=alpha) #'relu'
optimizer = 'adam' #SGD(learning_rate=0.01)

model = Sequential()
#first layer
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(64, 64, 1), 
                activation=activation, padding=padding))
model.add(AveragePooling2D(pool_size=(2, 2), padding='valid'))
model.add(Dropout(0.35))
# second layer
model.add(Conv2D(filters=128, kernel_size=(2,2), activation=activation))
model.add(AveragePooling2D(pool_size=(2, 2), padding='valid'))
model.add(Dropout(0.3))
# third layer
model.add(Conv2D(filters=256, kernel_size=(2,2), activation=activation))
model.add(MaxPool2D(pool_size=(2, 2), padding=padding))
model.add(Dropout(0.3))
# fourth layer
model.add(Conv2D(filters=256, kernel_size=(2,2), activation=activation))
model.add(MaxPool2D(pool_size=(2, 2), padding=padding))
model.add(Dropout(0.3))
# fifth layer
model.add(Conv2D(filters=512, kernel_size=(2,2), activation=activation))
model.add(Dropout(0.3))
# sixth layer
model.add(Conv2D(filters=512, kernel_size=(2,2), activation=activation))
model.add(Dropout(0.3))
# Dense layer
model.add(Flatten())
model.add(Dense(1024, activation=activation))
model.add(Dropout(0.3))
model.add(Dense(1024, activation=activation))
model.add(Dropout(0.25))
model.add(Dense(1, activation=activation))

model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

#model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
#model.add(AveragePooling2D(pool_size=(2, 2), padding='valid'))

In [ ]:
model.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=8, verbose=0, mode='min')
#mcp_save = ModelCheckpoint('weights.hdf5', save_best_only=True, monitor='val_loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

In [ ]:
del x_train #to make the memory available for training the model

# 7. Model fitting:

In [ ]:
model.fit(x_train_extra,y_train_extra,epochs=30,validation_data=(x_val,y_val), batch_size=800)
#model.fit(x_train_extra,y_train_extra,epochs=15,validation_data=(x_val,y_val), batch_size=1000,callbacks=[earlyStopping, mcp_save])

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss', 'val_loss']].plot()

In [ ]:
# Saving model 
model.save('tc_model') # the input to model.save() should a path that you would like to save the model to

In [ ]:
# Loading the pretrained model (if already available):
model = load_model(f'{directory}saved_model')

In [ ]:
predictions = model.predict(x_val)

In [ ]:
preds = pd.DataFrame(y_val.reshape(-1,1), columns=['truth'])
preds['model'] = predictions
preds.head()

In [ ]:
preds['diff'] = abs(preds['truth']-preds['model'])

In [ ]:
val['prediction'] = preds.model.values.astype(int)
val = val.drop(columns=['file_name'])
# val.to_csv('prediction_val_Radiant.csv')

In [ ]:
round(mean_squared_error(preds.model, preds.truth, squared=False),2)

In [ ]:
round(mean_absolute_error(preds.truth,preds.model),2)

In [ ]:
preds.groupby('truth').mean()['diff'].rolling(window=10).mean().plot()

# 8. Prediction for test period:

In [ ]:
# we need to delete the training data and release memory to be able to run the model for test period.
del x_train_extra

In [ ]:
x_test = np.load(f'{directory}/x_test.npy')
test_metadata  = pd.read_csv(f'{directory}/test_set_features.csv')

In [ ]:
test_pred = model.predict(x_test)
test_pred.shape

In [ ]:
test_metadata['prediction'] = test_pred[:,0]
test_metadata.to_csv('prediction_test_Radiant.csv')

In [ ]:
test_metadata['wind_speed'] = test_metadata.prediction.apply(lambda x: round(x))

In [ ]:
#saving submission
test_metadata[['image_id','wind_speed']].set_index('image_id').to_csv('Submission_Radiant.csv')